# 1. Experimental design

Suppose you are rummaging in an attic and come across an old weighted coin.  The coin is accompanied with an decaying pamphlet which states the coin is weighted to come up on one side 60% of the time.  From holding the coin, you can't tell which is the favored side.

Before you start flipping the coin, you wish to estimate the number of times you will need to flip the coin in order to be $99.85$% sure you have identified each side of the coin with the correct probability (i.e. $\text{erf}(3)$, or one-sided three standard deviations).

1. Perform an estimate using the central limit theorem.
2. Perform an estimate using Cramer's large deviation theorem.
3. Perform an estimate using numerical simulation.
4. When compared with the numerics, did the two theorems give over or under estimates?
5. (Non-rigorous question) What are some ideas you have on why the theorems differed from practice?  Don't feel like you have to be too rigorous in answering this question, but feel free to try to figure this out with some extra reading, some conjectures, or some additional numerical experimentation.  In particular, we are going to examine the descrepancy in Cramer's Theorem below.
  
*Hint:* If helpful, a restatement of Cramer's theorem states that 

* $\lim_{N\rightarrow\infty} \frac{1}{N} \log(\mathbf{P}(\bar{f}_N \geq a) = -\Lambda^*(a)$, where $\bar{f}_N$ is the sample mean, $a>\mathbb{E}[f]$, and $\Lambda^*(\lambda)$ is the Legendre transform of $\log(\mathbb{E}[e^{X\lambda}])$. Here the Legendre transform is the negative of how we defined it in week 2 of class (see week 2, day 2 notes an also the comment at the beginning of week 3, day 1 notes). 
* Also $\lim_{N\rightarrow\infty} \frac{1}{N} \log(\mathbf{P}(\bar{f}_N \leq a) = -\Lambda^*(a)$, where $\bar{f}_N$ is the sample mean, $a>\mathbb{E}[f]$, and $\Lambda^*(\lambda)$ is the Legendre transform of $\log(\mathbb{E}[e^{X\lambda}])$.

For more details see [here](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_theorem_(large_deviations)#:~:text=Cram%C3%A9r's%20theorem%20is%20a%20fundamental,by%20Harald%20Cram%C3%A9r%20in%201938.)

*Hint:* When performing numerical experiments, it will be easier to use the binomial distribution with $N$ trials than to run a Burnoulli experiment $N$ times.

# 2. Investigating Cramer's Theorem Numerically

To further investigate the discrepancies you found in Cramer's thoerem

1. Write a function that takes $N$ as an argument and generates a sample mean, $\bar{f}_N$, from a Bernoulli random variable with $p=0.4$ (you likely did something similar to above; if you haven't take this as a hint for the above problem).
2. Write a routine that calls the above function and samples $\bar{f}_N$.  Show that $\sqrt{N}(\bar{f}_N - \mathbb{E}(f))$ approaches a Guassian distribution both through a direct visualization and a quantile-quantile (QQ) plot.
  - A quantile–quantile (QQ) plot is a plot of the quantiles (i.e. the inverse of the cumulative distribution function) of two 1-dimensional distributions against one another. If the resulting curve is y = x, the distributions are the same. This is most often used when at least one of the distributions is empirical (i.e. a collection of samples) and you want to know how close those samples are to some specific distribution. Produce QQ plots to accompany your histograms.
  - A tool that might help is the notebook provided from week 2, day 2 when visualizing the central limit theorem.
3. Next, write a routine that constructs an estimate $Q_N$ of the probability $p_N = P(\bar{f}_N > 0.5)$ (to do this generate many samples of $\bar{f}_N$, much like you did in the previous exercise, but now for various values of $N$). 
4. Taking a log of the probabilities, plot the prediction of Cramer's theorem (i.e. $-n\gamma(\epsilon)$) against what you observe numerically.  Are your finding surprising?  Do they agree with or contradict Cramer's theorem? 

In [168]:
# potentially helpful ploting calls

def logDecay(x):
    return -flips*Legendre_Transform_at_oneHalf

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=rangeOfFlips, y=np.log(fractionOfWrongPredictions),
                    mode='lines+markers',
                    name='Fraction of wrong predictions(numerical)'))
fig.add_trace(go.Scatter(x=rangeOfFlips, y=[logDecay(flips) for flips in rangeOfFlips],
                    mode='lines',
                    name='Prediction from Cramer\'s theorem'))
fig.show()

# 3. Investigating Cramer's Theorem (Analytically)

The reason for the descrepancies above are due to sub-exponential terms.  For an illustration of this see [this answer](https://math.stackexchange.com/a/2366311) on math.stackexchange.  

We may approximate the scaled binomial distribution we have used above as a normal distribution and then use the bounds and estimates from the above post.  Note that $\bar{f}_N$ is approximately distributed as a normal distribution with mean $0.4$ and variance $\sigma^2 = \frac{0.4(1-0.4)}{\sqrt{N}}$.

Let $X\sim \mathcal{N}(0.4, \frac{0.4(1-0.4)}{\sqrt{N}})$. 

Analytically, $P(\bar{f}_N\approx X > 0.5) = \int_{0.5}^{\infty} \frac{1}{\sqrt{2 \pi} \sigma} e^{-(x-0.4)^2/(2 \sigma^2)} dx = \int_{0.1 \sqrt{n}/(0.4(1-0.4))}^{\infty} \frac{1}{\sqrt{2 \pi}} e^{-x^2/2} dx = \text{erf}(0.1 \sqrt{n}/(0.4(1-0.4))$

From the post above, note that $\text{erf}(a) < \frac{1}{a \sqrt{2 \pi}} e^{-a^2/2}$ (as a bonus, prove this through) and that this upper bound is fairly tight, i.e. $\text{erf}(a) \approx \frac{1}{a \sqrt{2 \pi}} e^{-a^2/2}$

1. Use Cramer's Theorem to estimate $\log(P(X > 0.5))$.
2. Confirm that Cramer's theorem agrees with $\log(\frac{1}{a \sqrt{2 \pi}} e^{-a^2/2})$ in the limit of large $N$, where $a = 0.1 \sqrt{n}/(0.4(1-0.4))$.
3. Examine at the relative error in probability between the estimate from Cramer's theorem and the analytic approximation of $\text{erf}(a)$.  What do you find?